In [2]:
# Why aren't my changesets similar
# figure out why no preds are being made
# Load dictionaries
from os import listdir
from os.path import isfile, join
import yaml
from tqdm import tqdm

# load rule dic 
rule_name = "/home/ubuntu/praxi/fall2019/hyb_rules.yaml"
with open(rule_name, 'r') as stream:
    rule_dic = yaml.safe_load(stream)

# load data (examples of chrony)
hyb_dir = "/home/ubuntu/praxi/fall2019/data/version_hybsets"
hybset_fnames = [f for f in listdir(hyb_dir) if isfile(join(hyb_dir, f))]

hyb_dics = [] 
for hyb_name in tqdm(hybset_fnames):
        full_hyb_name = hyb_dir + '/' + hyb_name
        with open(full_hyb_name, 'r') as stream:
            hyb_dic = yaml.safe_load(stream)
        hyb_dics.append(hyb_dic)
        
print(hyb_dics[0])

# Ultimate goal: Output application name & version, then list of all file paths NOT 
                #in the intersection for at least one changeset (union - intersection)


100%|██████████| 1300/1300 [01:19<00:00, 16.38it/s]

{'changes': ['/usr/share/man/man1/rrdtool.1.gz', '/usr/share/man/man3/librrd.3.gz', '/usr/share/doc/rrdtool/html/rrdgraph_libdbi.html', '/usr/share/doc/rrdtool/examples/perftest.pl.gz', '/usr/share/man/man1/cdeftutorial.1.gz', '/usr/share/doc/rrdtool/html/rrdthreads.html', '/var/cache/man/de/31234', '/usr/share/doc/rrdtool/html/rrdlastupdate.html', '/usr/share/doc/rrdtool/html/rpntutorial.html', '/usr/share/man/man1/rrdcreate.1.gz', '/usr/share/man/man1/rrdgraph_examples.1.gz', '/usr/share/man/man1/rrdcgi.1.gz', '/var/cache/man/sv/31234', '/usr/bin/rrdtool', '/usr/share/doc/rrdtool.dpkg-new/html/rrdfetch.html', '/usr/share/man/man1/rrdflushcached.1.gz', '/var/cache/man/pl/31234', '/usr/share/doc/rrdtool/copyright', '/usr/bin/rrdinfo.dpkg-new', '/usr/share/doc/rrdtool.dpkg-new/html/rrdgraph_examples.html', '/usr/share/doc/rrdtool.dpkg-new/copyright', '/usr/share/doc/rrdtool.dpkg-new/html/rrdinfo.html', '/usr/share/man/man1/rrdfetch.1.gz', '/var/lib/dpkg/tmp.ci/control', '/usr/share/man/

In [54]:
app_vers_names = []
app_names = []
for d in hyb_dics:
    cur_name = d["label"] + "_" + d["version"]
    cur_app = d["label"]
    if cur_name not in app_vers_names: 
        app_vers_names.append(cur_name)
    if cur_app not in app_names:
        app_names.append(cur_app)
app_vers_names.sort()
print(len(app_vers_names))
print(len(app_names))

130
45


In [56]:
# 130 specific app/versions
# 45 specific apps

changeset_lists = [ [] for i in range(130) ]

# this will contain a list of all the changesets for each app/label combo
for d in hyb_dics:
    cur_changes = d["changes"]
    cur_name = d["label"] + "_" + d["version"]
    idx = app_vers_names.index(cur_name)
    changeset_lists[idx].append(cur_changes)

#print(changeset_lists[0])
#print(len(changeset_lists[0]))

In [13]:
for i in range(130):
    print(len(changeset_lists[i]))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [14]:
new_changeset_lists = [ [] for i in range(130) ]
for i, changelist in enumerate(changeset_lists):
    intersection = set(changelist[0])
    for changes in changelist:
        intersection &= set(changes)
    for changes in changelist:
        new_changes = [x for x in changes if x not in intersection]
        new_changeset_lists[i].append(new_changes)
print(new_changeset_lists[0])

[['/var/cache/man/31928'], ['/usr/share/doc/amanda-server/examples.dpkg-new/chg-manual.conf', '/usr/share/doc/amanda-server/examples/chg-scsi.conf.dpkg-new', '/var/cache/man/31231', '/usr/share/doc/amanda-server/examples/README.dpkg-new', '/usr/share/doc/amanda-server/examples/amanda-S3.conf.dpkg-new', '/usr/share/doc/amanda-server/examples/inetd.conf.amandaserver.dpkg-new', '/usr/share/doc/amanda-server/examples/tapetypes.gz.dpkg-new', '/usr/share/doc/amanda-server/examples/disklist.dpkg-new', '/usr/share/doc/amanda-server/examples/dumptypes.gz.dpkg-new', '/usr/share/doc/amanda-server/examples/amanda-harddisk.conf.dpkg-new', '/usr/share/doc/amanda-server/NEWS.Debian.gz.dpkg-new', '/usr/share/doc/amanda-server/examples.dpkg-new/amanda.conf.gz', '/usr/share/doc/amanda-server/examples/amanda-single-tape.conf.dpkg-new', '/usr/share/doc/amanda-server/examples.dpkg-new/chg-scsi.conf', '/usr/share/doc/amanda-server/examples.dpkg-new/amanda-harddisk.conf', '/usr/share/doc/amanda-server/exampl

In [19]:
print(len(new_changeset_lists[0]))
zero_count = 0
for i, changelist in enumerate(new_changeset_lists):
    for changes in changelist:
        #print(len(changes))
        if(len(changes)==0):
            zero_count += 1
print("Zero count:", zero_count)

10
Zero count: 24


In [59]:
app_vers_intersections = [] 
app_vers_unions = []
app_vers_simscores = []
app_vers_differences = []
for i, changelist in enumerate(changeset_lists):
    cur_int = set(changelist[0])
    cur_union = set(changelist[0])
    for changes in changelist:
        cur_int &= set(changes)
        cur_union |= set(changes)
    cur_int = list(cur_int)
    cur_union = list(cur_union)
    app_vers_intersections.append(cur_int)
    app_vers_unions.append(cur_union)
    app_vers_simscores.append(round(len(cur_int)/len(cur_union), 2))
    cur_diff = [x for x in cur_union if x not in cur_int]
    app_vers_differences.append(cur_diff)
    

for diff in app_vers_differences:
    print(len(diff))

46
280
69
27
34
31
327
335
271
244
279
311
54
283
288
258
268
285
1018
2299
1200
440
726
452
113
263
271
340
287
260
423
270
111
275
275
120
276
286
77
273
283
274
346
307
818
1269
270
27
41
42
11904
6581
3
24
132
270
265
149
31
161
78
303
301
260
10
270
21
273
286
266
286
362
93
278
312
354
632
272
137
270
264
67
262
271
20
262
272
564
466
30
158
353
320
464
692
870
268
273
76
113
307
363
54
274
290
41
265
274
257
297
307
336
384
390
263
270
574
438
30
263
270
346
391
430
138
289
289
11
264
273


In [62]:
#app_vers_names
for name, inter, unio, simsco, diff in zip(app_vers_names, app_vers_intersections, app_vers_unions, app_vers_simscores, app_vers_differences):
    print(name, len(inter), len(unio), simsco, len(diff))


amanda-server_1:3.3.3-2ubuntu1 171 217 0.79 46
amanda-server_1:3.3.6-4.1 163 443 0.37 280
amanda-server_1:3.5.1-1build2 143 212 0.67 69
bacula-client_5.2.6+dfsg-9.1ubuntu3 1 28 0.04 27
bacula-client_7.0.5+dfsg-4ubuntu0.1 27 61 0.44 34
bacula-client_9.0.6-1build1 29 60 0.48 31
bind9_1:9.10.3.dfsg.P4-8ubuntu1.14 179 506 0.35 327
bind9_1:9.11.3+dfsg-1ubuntu1.8 201 536 0.38 335
bind9_1:9.9.5.dfsg-3ubuntu0.19 150 421 0.36 271
chrony_1.29-1ubuntu0.1 80 324 0.25 244
chrony_2.1.1-1ubuntu0.1 90 369 0.24 279
chrony_3.2-4ubuntu4.2 137 448 0.31 311
crash_7.0.3-3ubuntu4.5 29 83 0.35 54
crash_7.2.1-1ubuntu2 70 353 0.2 283
crash_7.2.3+real-1~16.04.1 35 323 0.11 288
cups_1.7.2-0ubuntu1.11 204 462 0.44 258
cups_2.1.3-4ubuntu0.9 226 494 0.46 268
cups_2.2.7-1ubuntu2.6 251 536 0.47 285
dovecot-core_1:2.2.22-1ubuntu2.10 975 1993 0.49 1018
dovecot-core_1:2.2.33.2-1ubuntu4.3 594 2893 0.21 2299
dovecot-core_1:2.2.9-1ubuntu2.6 919 2119 0.43 1200
dstat_0.7.2-3build1 167 607 0.28 440
dstat_0.7.2-4 175 901 0.19 7

In [37]:
for inter in app_vers_intersections:
    print(len(inter))
    #print("\n\n")

171
163
143
1
27
29
179
201
150
80
90
137
29
70
35
204
226
251
975
594
919
167
175
252
25
33
58
81
141
183
206
232
51
63
85
59
104
127
56
64
102
87
74
128
670
718
754
1
62
60
4468
7875
0
1
50
86
61
199
224
226
55
87
108
47
39
72
69
95
124
144
145
127
50
81
142
420
460
100
27
60
35
33
43
68
10
35
60
142
191
37
183
193
222
392
312
223
61
86
55
95
114
102
104
121
166
34
41
62
122
147
178
70
73
99
42
67
43
237
29
37
62
82
112
151
46
46
78
24
37
62


In [44]:
from itertools import combinations
from statistics import mean

# Jaccard Index
def jac(s1,s2):
    """the jaccard index between 2 sets"""
    s_union = s1.union(s2)
    s_inter = s1.intersection(s2)

    len_union = len(s_union)
    if not len_union:
        return 0

    return len(s_inter)*1.0/len_union

def jac_full(list0, list1):
    """Comparison taking into account keys and items"""
        
    set_keys0 = set(list0)
    set_keys1 = set(list1)

    key_commonality = jac(set_keys0, set_keys1)

    #ok, so now we give a proximity on key match, even if key + data dont match
    return key_commonality  #*.9 + 0.1 * item_commonality

def assess_sim(changes_list):
    # compare all dictionaries in the list to each other, return
    # average jaccard index
    combos = list(combinations(changes_list, 2))
    jac_scores = [jac_full(i[0], i[1]) for i in combos]
    #print(jac_scores)
    return mean(jac_scores)

In [45]:
print(len(changeset_lists))
print(len(changeset_lists[0]))

130
10


In [49]:
app_vers_jacscores = [] 
for changes_list in changeset_lists:
    #print(len(changes_list))
    cur_jac = round(assess_sim(changes_list),2)
    app_vers_jacscores.append(cur_jac)
    print(cur_jac)
#print(app_vers_jacscores)

0.9
0.73
0.87
0.42
0.78
0.82
0.73
0.75
0.73
0.61
0.61
0.68
0.73
0.55
0.43
0.78
0.81
0.81
0.74
0.66
0.69
0.67
0.64
0.81
0.58
0.38
0.52
0.61
0.7
0.78
0.66
0.81
0.71
0.53
0.6
0.74
0.64
0.68
0.8
0.54
0.64
0.59
0.6
0.66
0.76
0.66
0.93
0.5
0.86
0.88
0.71
0.8
0.01
0.44
0.66
0.61
0.53
0.77
0.97
0.79
0.78
0.57
0.63
0.47
0.95
0.57
0.91
0.62
0.69
0.73
0.72
0.68
0.74
0.6
0.69
0.77
0.7
0.65
0.53
0.53
0.4
0.76
0.45
0.56
0.66
0.4
0.52
0.6
0.76
0.86
0.8
0.68
0.78
0.74
0.72
0.72
0.53
0.61
0.78
0.77
0.64
0.63
0.87
0.68
0.72
0.81
0.43
0.53
0.69
0.69
0.73
0.51
0.49
0.56
0.44
0.55
0.61
0.68
0.83
0.41
0.53
0.54
0.58
0.63
0.63
0.49
0.59
0.86
0.42
0.53


In [52]:
app_vers_simscores.sort()
for score in app_vers_simscores:
    print(score)

0.0
0.04
0.04
0.04
0.07
0.11
0.11
0.12
0.12
0.12
0.12
0.13
0.14
0.14
0.14
0.15
0.16
0.16
0.17
0.18
0.18
0.18
0.18
0.18
0.18
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.19
0.2
0.2
0.2
0.2
0.2
0.2
0.21
0.21
0.21
0.22
0.22
0.22
0.23
0.24
0.24
0.24
0.24
0.24
0.25
0.25
0.26
0.26
0.26
0.26
0.26
0.27
0.27
0.27
0.27
0.27
0.28
0.29
0.29
0.3
0.31
0.31
0.31
0.31
0.31
0.31
0.32
0.33
0.33
0.33
0.33
0.33
0.33
0.34
0.35
0.35
0.35
0.35
0.35
0.35
0.36
0.36
0.36
0.36
0.37
0.37
0.38
0.41
0.41
0.41
0.42
0.42
0.42
0.43
0.44
0.44
0.45
0.45
0.46
0.46
0.46
0.46
0.47
0.48
0.49
0.49
0.54
0.54
0.54
0.55
0.57
0.58
0.59
0.6
0.66
0.67
0.69
0.74
0.77
0.79
0.8
0.88


In [65]:
worst_name = "mariadb-server_10.0.38-0ubuntu0.16.04.1"
idx = app_vers_names.index(worst_name)
print(app_vers_differences[idx])
print(changeset_lists[idx])

['/var/log/journal/90c213c3662a46589e220665c75c823f/system.journal', '/var/log/syslog', '/var/log/auth.log']
[[], [], [], [], ['/var/log/auth.log'], [], [], [], [], ['/var/log/journal/90c213c3662a46589e220665c75c823f/system.journal', '/var/log/auth.log', '/var/log/syslog']]


In [67]:
best_name = "mc_3:4.8.15-2"
idx = app_vers_names.index(best_name)
print(app_vers_differences[idx])
#print(changeset_lists[idx])

['/var/lib/apt/extended_states.b1VSAn', '/var/lib/apt/extended_states.JjqE0M', '/var/lib/apt/extended_states.N3QkTR', '/var/cache/apt/pkgcache.bin.OEwmj1', '/var/cache/apt/pkgcache.bin.JHHRPA', '/var/lib/update-notifier/tmp.CwXhoR3BRC', '/var/lib/apt/extended_states.66j28t', '/var/lib/apt/extended_states.ZVTPfO', '/var/cache/apt/pkgcache.bin.I9xkUA', '/var/lib/update-notifier/tmp.7SQtstdGrD', '/var/lib/apt/extended_states.KMF94b', '/var/lib/update-notifier/tmp.BwEDGQavDN', '/var/lib/apt/extended_states.m8bMOb', '/var/lib/update-notifier/tmp.tEFf082GzD', '/var/lib/update-notifier/tmp.aBy4xzvGhj', '/var/lib/update-notifier/tmp.3aWY9ivKuz', '/var/lib/update-notifier/tmp.A87HmuekIy', '/var/cache/apt/pkgcache.bin.AOlHyp', '/var/cache/apt/pkgcache.bin.tueQuc', '/var/cache/apt/pkgcache.bin.FoBjMX', '/var/cache/apt/pkgcache.bin.eyT42a', '/var/lib/update-notifier/tmp.prOyvYzNyo', '/var/log/syslog', '/var/cache/apt/pkgcache.bin.GX9tbd', '/var/lib/apt/extended_states.ZnvrtR', '/var/lib/apt/extend

In [17]:
X = []
y = []
for d in hyb_dics:
    if d["label"] == "mc":
        X.append(d["changes"])
        y.append(d["version"])

labels = list(set(y))
partitioned_X = [ [] for i in range(len(labels)) ]
unions = []
print(labels)


['3:4.8.15-2', '3:4.8.11-1', '3:4.8.19-1']


In [18]:
for cur_changes, cur_label in zip(X,y):
    idx = labels.index(cur_label)
    partitioned_X[idx].append(cur_changes)
    
print(len(partitioned_X))
print(len(partitioned_X[0]))

#print(partitioned_X[0][0])

3
10


In [20]:
for partition in partitioned_X:
    cur_union = set(partition[0])
    for cur_changes in partition:
        #print(cur_changes)
        cur_union |= set(cur_changes)
    unions.append(list(cur_union))